**Download IODv4 Toolkit**

In [3]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
!pip install -r OIDv4_ToolKit/requirements.txt

fatal: destination path 'OIDv4_ToolKit' already exists and is not an empty directory.


**Download dataset from OpenImages using OID Toolkit**

In [0]:
!python3 OIDv4_ToolKit/main.py downloader --classes Orange --type_csv all


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Orange.

Orange
    [INFO] | Downloading all images.
    [INFO] | [INFO] Found 900 online images for train.
    [INFO] | All images already downloaded.
    [INFO] | Creating labels for Orange of t

**Install required packages for converting OIDv4 labels to a single CSV file**

**Install required packages for creating TFRecord file**

**Clone the to begin the training process**
